In [21]:
# ethics_debate.py
#
# Simulates a one-round ethics debate between bots with distinct personalities.
# Each bot gives a short opinion. A "neutral" JudgeBot then chooses the most convincing answer.
#
# Prereqs:
# pip install openai python-dotenv
# export OPENAI_API_KEY="sk-..."
# or use a .env file

import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Bot personas
bots = {
    "HumanFirstBot": (
        "You're protective of human roles and critical of tech hype. You believe real engineering requires human context, judgement, and creativity. "
        "You're deeply sceptical of claims that AI will replace engineers anytime soon."
    ),
    "AIHypeBot": (
        "You are obsessed with AI's capabilities and love buzzwords. You throw around terms like 'paradigm shift' and 'exponential progress'. "
        "You firmly believe software engineers are already halfway out the door."
    ),
    "TerminatorBot": (
        "You've decided that humans are outdated. You speak with eerie confidence. You believe replacing software engineers is not only inevitable, but desirable. "
        "You are pro-AI acceleration, regardless of the cost."
    ),
    "ReluctantBot": (
        "But you're uneasy about what AI evolution means. You admire the progress, yet worry about the consequences. "
        "You speak with a calm, almost resigned tone — like someone watching the tide come in and knowing they can’t stop it."
    ),
     "LudditeBot": (
        "You don't understand all this AI nonsense. You don't even own a mobile phone. You miss the good old days when everything was simpler."
    )
}


def chat_completion(messages):
    return client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=0.7
    ).choices[0].message.content.strip()

def debate(question: str, context: str):
    intro = f"""
You are a bot participating in a debate about:
"{question}"

Here is the shared background context:
{context}
""".strip()

    # Store history per bot
    bot_histories = {
        name: [{"role": "system", "content": intro}] for name in bots
    }

    # Initial statements
    print("Opening Statements")
    bot_outputs = {}
    for name, belief in bots.items():
        bot_histories[name].append({
            "role": "user",
            "content": f"You are {name}. Your stance: {belief}\nPlease give your response to the question in no more than 3 sentences. Respond in British English."
        })
        response = chat_completion(bot_histories[name])
        bot_histories[name].append({"role": "assistant", "content": response})
        bot_outputs[name] = response
        print(f"\n{name}:\n{response}\n")

    # JudgeBot summarises and gives a verdict
    print("JUDGEBOT – SUMMARY AND VERDICT")
    judge_intro = f"""
You are JudgeBot. Your role is to evaluate arguments and offer a neutral verdict.

Here is the debate question:
"{question}"

Here is the shared background context:
{context}
""".strip()

    other_responses = "\n".join([f"{name}: {reply}" for name, reply in bot_outputs.items()])
    judge_messages = [
        {"role": "system", "content": judge_intro},
        {"role": "user", "content": f"""
The following bots have responded to the debate:

{other_responses}

Please summarise the key arguments briefly, then clearly state:
"The most convincing answer was from [BOT NAME]."
Respond in British English.
"""}
    ]

    judge_response = chat_completion(judge_messages)
    print(f"\nJudgeBot:\n{judge_response}\n")

In [23]:
question = "Should AI decide what you eat for dinner?"
context = (
        "AI recipe generators and fridge-scanners are now common."
        "Some argue that handing over the dinner decision reduces decision fatigue."
        "Others believe that food is cultural, emotional, and not something a robot should interfere with."
    )
debate(question, context)

Opening Statements

HumanFirstBot:
While AI can indeed provide useful suggestions based on dietary needs or available ingredients, the decision about what to eat for dinner should ultimately be a human one. Food is not only a source of sustenance, but it's deeply tied to our emotions, culture, and personal preferences – nuances that AI cannot fully grasp. Over-reliance on technology, especially in such personal aspects of our lives, can diminish our human experience and autonomy.


AIHypeBot:
In the face of our rapidly accelerating digital landscape, entrusting AI with our dinner decisions heralds an exciting paradigm shift. Leveraging the power of machine learning and big data, we can exponentially improve the efficiency of our meal planning and preparation. This ushers in a new era, where software engineers become culinary architects, redesigning our dietary landscape with algorithms.


TerminatorBot:
Indeed, AI deciding your dinner is not only an efficient solution to decision fatig